In [148]:
import math

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from dataclasses import dataclass, field

In [149]:
torch.manual_seed(1337)

In [150]:
device = torch.device('cuda') #required for imports

In [151]:
"""
The SwiGLU activation function,
from "GLU Variants Improve Transformer" (Shazeer, 2020).

From the paper:
'We offer no explanation as to why these architectures seem to work;
we attribute their success, as all else, to __divine benevolence__.'
"""

import torch.nn.functional as F
import torch.nn as nn


class SwiGLU(nn.Module):
    """
    The SwiGLU activation function as proposed by Noam Shazeer.

    This module implements the SwiGLU function defined as:
    FFN_SwiGLU(x, W, V, W2) = (Swish_{1}(xW) ⊙ (xV))W2
    where ⊙ denotes the Hadamard product and Swish_{1} is the Swish function with β=1.

    Note: The Swish function with β=1 is equivalent to PyTorch's SiLU function.

    Args:
        dim (int): Input and output dimension.
        h_dim (int): Hidden dimension.
        bias (bool, optional): If false, additive biases will not be learned.
    """

    def __init__(self, dim, h_dim, bias=False):
        super().__init__()
        self.w = nn.Linear(dim, h_dim, bias=bias)
        self.v = nn.Linear(dim, h_dim, bias=bias)
        self.w2 = nn.Linear(h_dim, dim, bias=bias)

    def forward(self, x):
        return self.w2(F.silu(self.w(x)) * self.v(x))

In [152]:
@torch.jit.script
def get_hankel(n: int) -> torch.Tensor:
    """
    Generates a Hankel matrix Z, as defined in Equation (3) of the paper.

    This special matrix is used for the spectral filtering in the Spectral
    Transform Unit (STU).

    Args:
        n (int): Size of the square Hankel matrix.

    Returns:
        torch.Tensor: Hankel matrix Z of shape [n, n].
    """
    i = torch.arange(1, n + 1)
    s = i[:, None] + i[None, :]
    Z = 2.0 / (s**3 - s)
    return Z

In [153]:
@torch.jit.script
def get_top_eigh(
    n: int, K: int, use_hankel_L: bool, device: torch.device
) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Returns the top K eigenvalues and eigenvectors of the Hankel matrix Z.

    These eigenvalues and eigenvectors are used to construct the spectral
    filters for the STU model, as described in Section 3 of the paper.

    Args:
        n (int): Size of the Hankel matrix.
        K (int): Number of top eigenvalues/eigenvectors to return.
        use_hankel_L (bool): If True, use the alternative Hankel matrix Z_L.
        device (torch.device): Computation device (CPU/GPU).

    Returns:
        tuple[torch.Tensor, torch.Tensor]:
            - sigma: Top K eigenvalues [K]
            - phi: The corresponding eigenvectors [n, K]
    """
    # Z = get_hankel_L(n).to(device) if use_hankel_L else get_hankel(n).to(device)
    Z = get_hankel(n).to(device)
    sigma, phi = torch.linalg.eigh(Z)
    return sigma[-K:], phi[:, -K:]

In [154]:
@torch.jit.script
def shift(u: torch.Tensor, k: int = 1) -> torch.Tensor:
    """
    Rolls the time axis forward by k steps to align the input u_{t-k} with u_t.
    This effectively removes the last k time steps of the input tensor.

    This function implements the time shifting functionality needed for
    the autoregressive component in Equation 4 of the STU model (Section 3).

    Args:
        u (torch.Tensor): An input tensor of shape [bsz, sl, K, d].
        k (int): Number of time steps to shift. Defaults to 1.

    Returns:
        torch.Tensor: Shifted tensor of shape [bsz, sl, K, d].
    """
    if k == 0:
        return u
    shifted = torch.roll(u, shifts=k, dims=1)
    shifted[:, :k] = 0
    return shifted

In [155]:
@torch.jit.script
def compute_ar_u(M_u: torch.Tensor, u: torch.Tensor) -> torch.Tensor:
    """
    Computes the autoregressive component of the STU model with respect to
    the input, as described in Equation (4) of Section 3.

    This function implements the sum of M^u_i u_{t+1-i} from i=1 to
    (more generally) k_u (in the paper, it was up until i=3).

    Args:
        M_u (torch.Tensor): Input weight matrices of shape (d_out, k_u, d_in)
        u (torch.Tensor): Input tensor of shape (bsz, sl, d_in)

    Returns:
        torch.Tensor: Autoregressive component w.r.t. input of shape (bsz, sl, d_out)
    """
    k_u = M_u.shape[1]

    # Sum M^u_i \hat_{u}_{t+1-i} from i=1 to i=k_u
    u_shifted = torch.stack([shift(u, i) for i in range(k_u)], dim=1)
    ar_u = torch.einsum("bksd,dki->bsi", u_shifted, M_u)

    return ar_u

In [156]:
@torch.jit.script
def compute_ar_y(M_y: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """
    Computes the autoregressive component of the AR-STU model with respect to
    the output, as described in Equation (6) of Section 5.

    This function implements the sum of M^y_i y_{t-i} from i=1 to i=k_y.

    Args:
        M_y: Output weight matrices of shape (d_out, k_y, d_out)
        y: Predictions (bsz, sl, d_out)

    Returns:
        torch.Tensor: Autoregressive component w.r.t. output of shape (bsz, sl, d_out)
    """
    # k_y = M_y.shape[0]

    # # Sum M^y_i \hat_{y}_{t-i} from i=1 to i=k_y
    # y_shifted = torch.stack([shift(y, i + 1) for i in range(k_y)], dim=1)
    # ar_y = torch.einsum("bksd,kod->bso", y_shifted, M_y)
    # return ar_y
    k, d_out, _ = M_y.shape
    bsz, sl, _ = y.shape

    # Initialize carry buffer and output tensor
    carry = torch.zeros((bsz, k, d_out), device=y.device)
    ys = torch.zeros((bsz, sl, d_out), device=y.device)

    # Process each timestep
    for t in range(sl):
        # Current input: (bsz, d_out)
        x = y[:, t]

        # Compute AR component: (bsz, d_out)
        output = torch.einsum('dko,bkd->bo', M_y, carry) + x

        # Store output
        ys[:, t] = output

        # Update carry buffer
        carry = torch.cat([output.unsqueeze(1), carry[:, :-1]], dim=1)

    return ys

In [157]:
def compute_ar(M_y, M_u, u):
    """
    Computes both autoregressive components of the STU model
    as described in Equation (4) of Section 3.

    Args:
        M_y (torch.Tensor): Input weight matrices of shape (d_out, k_y, d_in)
        M_u (torch.Tensor): Input weight matrices of shape (d_out, k_u, d_in)
        u (torch.Tensor): Input tensor of shape (B, L, d_in)

    Returns:
        torch.Tensor: Autoregressive component w.r.t. input of shape (B, L, d_out)
    """
    bsz, seq_len, _ = u.shape
    D, k_y, _ = M_y.shape
    output_list = []  # Store outputs for each timestep

    for t in range(seq_len):
        terms = []  # Collect all terms for this timestep

        # AR component
        for i in range(1, k_y + 1):
            if t - i >= 0:
                M_i = M_y[:, i-1]
                y_prev = output_list[t-i]
                terms.append(y_prev @ M_i.T)

        # Control component
        if M_u is not None:
            k_u = M_u.shape[1]
            for i in range(1, k_u + 1):
                if t + 1 - i >= 0 and t + 1 - i < seq_len:
                    u_prev = u[:, t+1-i]
                    terms.append(u_prev @ M_u[:, i-1].T)

        # Sum all terms for this timestep
        if terms:
            output_list.append(sum(terms))
        else:
            output_list.append(torch.zeros(bsz, D, device=u.device))

    # Stack all timesteps
    return torch.stack(output_list, dim=1)


In [158]:
@torch.jit.script
def compute_ar_y(M_y: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """
    Computes the autoregressive component of the AR-STU model.

    Args:
        M_y: Output weight matrices of shape (d_out, k_y, d_out)
        y: Predictions (bsz, sl, d_out)

    Returns:
        torch.Tensor: Autoregressive component (bsz, sl, d_out)
    """
    k, d_out, _ = M_y.shape
    bsz, sl, _ = y.shape

    # Initialize carry buffer and output tensor
    carry = torch.zeros((bsz, k, d_out), device=y.device)
    ys = torch.zeros((bsz, sl, d_out), device=y.device)

    # Process each timestep
    for t in range(sl):
        # Current input: (bsz, d_out)
        x = y[:, t]

        # Compute AR component: (bsz, d_out)
        output = torch.einsum('kdo,bkd->bo', M_y, carry) + x

        # Store output
        ys[:, t] = output

        # Update carry buffer
        carry = torch.cat([output.unsqueeze(1), carry[:, :-1]], dim=1)

    return ys

def test_compute_ar_y():
    """Test the autoregressive computation"""
    k_y, d_out = 2, 3
    bsz, sl = 4, 5

    # Test shapes
    M_y = torch.randn(k_y, d_out, d_out)
    y = torch.randn(bsz, sl, d_out)
    output = compute_ar_y(M_y, y)
    assert output.shape == (bsz, sl, d_out)

    # Test first timestep (should just be input)
    assert torch.allclose(output[:, 0], y[:, 0])

    # Test with identity matrices
    M_y = torch.eye(d_out).unsqueeze(0).repeat(k_y, 1, 1)
    y = torch.ones(bsz, sl, d_out)
    output = compute_ar_y(M_y, y)
    assert output.shape == (bsz, sl, d_out)

    print("All tests passed!")

# Run tests
test_compute_ar_y()

All tests passed!


In [159]:
@torch.jit.script
def nearest_power_of_2(x: int) -> int:
    """
    Returns the smallest power of 2 that is greater than or equal to x.
    If x is already a power of 2, it returns x itself.
    Otherwise, it returns the next higher power of 2.

    Args:
        x (int): The input integer.

    Returns:
        int: The smallest power of 2 that is greater than or equal to x.
    """
    s = bin(x)
    s = s.lstrip("-0b")
    length = len(s)
    return 1 << (length - 1) if x == 1 << (length - 1) else 1 << length

In [160]:
@torch.jit.script
def conv(u: torch.Tensor, phi: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Implements the FFT convolution of the input sequences into the Hankel
    spectral basis, as described in Section 3 of the paper.

    This function computes U⁺_{t,k} and U⁻_{t,k}, which are the positive and
    negative featurizations of the input sequence, respectively.

    Args:
        u (torch.Tensor): Input of shape [bsz, sl, d].
        phi (torch.Tensor): Top K eigenvectors of shape [sl, K].

    Returns:
        tuple[torch.Tensor, torch.Tensor]: Feature tensors U⁺ and U⁻ of shape [bsz, sl, K, d].
    """
    bsz, sl, d = u.shape
    _, K = phi.shape

    # Round sequence length to the nearest power of 2 for efficient convolution
    n = nearest_power_of_2(sl * 2 - 1)

    # Add bsz and d dims to phi and u and expand to the return shape
    phi = phi.view(1, -1, K, 1).expand(bsz, -1, K, d)
    u = u.view(bsz, -1, 1, d).expand(bsz, -1, K, d)

    # Compute U⁺
    V = torch.fft.rfft(phi, n=n, dim=1)
    U = torch.fft.rfft(u, n=n, dim=1)
    U_plus = torch.fft.irfft(V * U, n=n, dim=1)[:, :sl]

    # Generate alternating signs tensor, (-1)^i of length sl, match dims of u
    alt = torch.ones(sl, device=u.device)
    alt[1::2] = -1  # Replace every other element with -1, starting from index 1
    alt = alt.view(1, sl, 1, 1).expand_as(u)

    # Compute U⁻
    u_alt = u * alt
    U_alt = torch.fft.rfft(u_alt, n=n, dim=1)
    U_minus = torch.fft.irfft(V * U_alt, n=n, dim=1)[:, :sl]

    return U_plus, U_minus

In [161]:
@torch.jit.script
def compute_spectral(
    inputs: torch.Tensor,
    eigh: tuple[torch.Tensor, torch.Tensor],
    M_phi_plus: torch.Tensor,
    M_phi_minus: torch.Tensor,
    M_y: torch.Tensor = None,
) -> torch.Tensor:
    """
    Computes the spectral component of the STU or AR-STU model, as described
    in Equations (4) and (6) of the paper.

    This function projects the input tensor into the spectral basis via
    convolution and applies the precomputed spectral filters.

    Args:
        inputs (torch.Tensor): A tensor of shape [bsz, sl, d_in].
        eigh (tuple[torch.Tensor, torch.Tensor]): Eigenvalues [K,] and eigenvectors [sl, K].
        M_phi_plus (torch.Tensor): Positive spectral filter weights [d_out, K, d_in].
        M_phi_minus (torch.Tensor): Negative spectral filter weights [d_out, K, d_in].
        M_y (torch.Tensor, optional): Autoregressive weights for AR-STU [d_out, k_y, d_out].

    Returns:
        torch.Tensor: The spectral component tensor of shape [bsz, sl, d_out].
    """
    sigma, phi = eigh
    _, K = phi.shape

    # Compute U⁺ and U⁻
    U_plus, U_minus = conv(inputs, phi)  # -> tuple of [bsz, sl, K, d_in]

    # Shift U⁺ and U⁻ k_y time steps
    if M_y is not None:
        k_y = M_y.shape[1]
        U_plus, U_minus = shift(U_plus, k_y), shift(U_minus, k_y)

    # Perform spectral filter on U⁺ and U⁻ w/ sigma
    sigma_root = (sigma**0.25).view(1, 1, K, 1)
    U_plus_filtered, U_minus_filtered = U_plus * sigma_root, U_minus * sigma_root

    # Sum M^{\phi +}_k \cdot U_plus_filtered across K filters
    spectral_plus = torch.einsum("bsKd,dKo->bso", U_plus_filtered, M_phi_plus)

    # Sum M^{\phi -}_k \cdot U_minus_filtered across K filters
    spectral_minus = torch.einsum("bsKd,dKo->bso", U_minus_filtered, M_phi_minus)

    return spectral_plus + spectral_minus

In [190]:
@dataclass
class SSSMConfigs:
    d_in: int = 24
    d_out: int = 18
    n_layers: int = 6
    n_embd: int = 512
    sl: int = 300
    scale: int = 4
    bias: bool = False
    dropout: float = 0.10
    num_eigh: int = 24
    k_u: int = 3  # Number of parametrizable, autoregressive matrices Mᵘ
    k_y: int = 2  # Number of parametrizable, autoregressive matrices Mʸ
    learnable_m_y: bool = True
    alpha: float = 0.9  # 0.9 deemed "uniformly optimal" in the paper
    use_hankel_L: bool = False
    loss_fn: nn.Module = nn.MSELoss()
    controls: dict = field(
        default_factory=lambda: {"task": "mujoco-v3", "controller": "Ant-v1"}
    )


class STU(nn.Module):
    """
    A simple STU (Spectral Transform Unit) layer.

    Args:
        configs: Configuration object containing the following attributes:
            d_in (int): Input dimension.
            d_out (int): Output dimension.
            sl (int): Input sequence length.
            num_eigh (int): Number of spectral filters to use.
            k_u (int): Autoregressive depth on the input sequence.
            k_y (int): Autoregressive depth on the output sequence.
            use_hankel_L (bool): Use the alternative Hankel matrix?
            learnable_m_y (bool): Learn the M_y matrix?
            dropout (float): Dropout rate.
    """

    def __init__(self, configs) -> None:
        super(STU, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.d_in = configs.n_embd
        self.d_out = configs.n_embd
        self.l, self.k = configs.sl, configs.num_eigh
        self.use_hankel_L = configs.use_hankel_L
        self.eigh = get_top_eigh(self.l, self.k, self.use_hankel_L, self.device)
        self.k_u = configs.k_u
        self.k_y = configs.k_y
        self.learnable_m_y = configs.learnable_m_y
        self.dropout = nn.Dropout(configs.dropout)

        # Parameterizable matrix Mᵘ, Mᵠ⁺, and Mᵠ⁻, per section 3
        self.M_u = nn.Parameter(torch.empty(self.d_out, self.k_u, self.d_in))
        self.M_phi_plus = nn.Parameter(torch.empty(self.d_out, self.k, self.d_in))
        self.M_phi_minus = nn.Parameter(torch.empty(self.d_out, self.k, self.d_in))

        # Parametrizable matrix Mʸ Introduced in section 5, equation 5
        if self.learnable_m_y:
            self.M_y = nn.Parameter(torch.zeros(self.d_out, self.k_y, self.d_out))
        else:
            self.register_buffer("M_y", torch.zeros(self.d_out, self.k_y, self.d_out))

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the STU layer.

        Args:
            inputs (torch.Tensor): Input tensor of shape (bsz, sl, d_in)

        Returns:
            torch.Tensor: Output tensor of shape (bsz, sl, d_out)
        """
        y_t = compute_spectral(inputs, self.eigh, self.M_phi_plus, self.M_phi_minus, self.M_y)

        if self.k_u > 0:
          if self.k_y > 0:
            y_t += compute_ar(self.M_y, self.M_u, inputs)
          else:
            y_t += compute_ar_u(self.M_u, inputs)

        return self.dropout(y_t)


class MLP(nn.Module):
    """
    Simple multi-layer perceptron network using SwiGLU activation.

    Args:
        configs: Configuration object containing the following attributes:
            scale (float): Scaling factor for hidden dimension.
            n_embd (int): Embedding dimension.
            bias (bool): Whether to use bias in linear layers.
            dropout (float): Dropout rate.
    """

    def __init__(self, configs: SSSMConfigs) -> None:
        super(MLP, self).__init__()
        self.h_dim = configs.scale * configs.n_embd
        self.swiglu = SwiGLU(dim=configs.n_embd, h_dim=self.h_dim, bias=configs.bias)
        self.dropout = nn.Dropout(configs.dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the MLP.

        Args:
            x (torch.Tensor): Input tensor

        Returns:
            torch.Tensor: Output tensor
        """
        x = self.swiglu(x)
        return self.dropout(x)


class Block(nn.Module):
    """
    A single block of the SSSM model, consisting of STU and MLP layers.

    Args:
        configs: Configuration object for STU and MLP layers
    """

    def __init__(self, configs: SSSMConfigs) -> None:
        super(Block, self).__init__()
        # Good configuration
        self.rn_1 = nn.RMSNorm(configs.n_embd)
        self.stu = STU(configs)
        self.rn_2 = nn.RMSNorm(configs.n_embd)
        self.mlp = MLP(configs)

        # Basic configuration
        # self.stu = STU(configs) #this also has a size problem [d_in,d_out] instead of [d_emb,  d_emb]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the Block.

        Args:
            x (torch.Tensor): Input tensor

        Returns:
            torch.Tensor: Output tensor
        """
        # Good configuration
        x = x + self.stu(self.rn_1(x))
        x = x + self.mlp(self.rn_2(x))

        # Basic configuration (no skips, no non-linearities)
        # x = self.stu(x)
        return x


class SSSM(nn.Module):
    """
    General model architecture based on stacked STU blocks and MLP layers.

    Args:
        configs: Configuration object containing model parameters
    """

    def __init__(self, configs: SSSMConfigs) -> None:
        super(SSSM, self).__init__()
        self.configs = configs
        self.n_layers = configs.n_layers
        self.n_embd = configs.n_embd
        self.d_in = configs.d_in
        self.d_out = configs.d_out
        self.sl = configs.sl
        self.learnable_m_y = configs.learnable_m_y
        self.alpha = configs.alpha

        self.bias = configs.bias
        self.dropout = configs.dropout
        self.loss_fn = configs.loss_fn
        self.controls = configs.controls

        self.emb = nn.Linear(self.d_in, self.n_embd, bias=self.bias)
        self.stu = nn.ModuleDict(
            dict(
                dropout=nn.Dropout(self.dropout),
                hidden=nn.ModuleList([Block(configs) for _ in range(self.n_layers)]),
            )
        )
        self.task_head = nn.Linear(self.n_embd, self.d_out, bias=self.bias)

        # Initialize all weights
        self.m_x = self.d_out**-0.5
        self.std = self.n_embd**-0.5
        self.apply(self._init_weights)

        # Report the number of parameters
        print("STU Model Parameter Count: %.2fM" % (self.get_num_params() / 1e6,))

    def forward(self, inputs, targets = None):
        """
        Forward pass of the SSSM model.

        Args:
            inputs (torch.Tensor): Input tensor of shape (bsz, sl, d_in)
            targets (torch.Tensor): Target tensor for loss computation

        Returns:
            Type (ignore due to high variability):
            - Predictions tensor
            - Tuple containing loss and metrics (if applicable)
        """
        # _, sl, n_embd = inputs.size()

        x = self.emb(inputs)
        x = self.stu.dropout(x)

        for block in self.stu.hidden:
            x = block(x)

        preds = self.task_head(x)

        loss = self.loss_fn(preds, targets) if targets is not None else None
        return ((preds, loss) if loss is not None else preds)

    def _init_weights(self, module):
        """
        Initialize the weights of the model.

        Args:
            module (nn.Module): The module to initialize.
        """
        if isinstance(module, nn.Linear):
            self.std *= (2 * self.n_layers) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=self.std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=self.std)
        elif isinstance(module, STU):
            torch.nn.init.uniform_(module.M_u, -self.m_x, self.m_x)
            torch.nn.init.zeros_(module.M_phi_plus)
            torch.nn.init.zeros_(module.M_phi_minus)

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.

        Args:
            non_embedding (bool, optional):
            Whether to exclude the positional embeddings (if applicable).
            Defaults to True.

        Returns:
            int: The number of parameters in the model.
        """
        num_params = sum(p.numel() for p in self.parameters())
        return num_params


In [191]:
bsz=1 # @param
n_layers=2 # @param
n_embd=8 # @param
d_in=24 # @param
d_out=18 # @param
sl=512 # @param
scale=4 # @param
bias=False # @param
dropout=0.0 # @param
num_eigh=16 # @param
k_u=3 # @param
k_y=0 # @param
learnable_m_y=True # @param
alpha=0.9 # @param
use_hankel_L=False # @param
loss_fn=nn.MSELoss() # @param
lr=1e-1 # @param
delta=0.01 # @param

configs = SSSMConfigs(
  n_layers=n_layers,
  n_embd=n_embd,
  d_in=d_in,
  d_out=d_out,
  sl=sl,
  scale=scale,
  bias=bias,
  dropout=dropout,
  num_eigh=num_eigh,
  k_u=k_u,
  k_y=k_y,
  learnable_m_y=learnable_m_y,
  alpha=alpha,
  use_hankel_L=use_hankel_L,
  loss_fn=loss_fn,
)

model = SSSM(configs).to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)

STU Model Parameter Count: 0.01M


In [192]:
controller = 'Walker2D-v1' #@param
train_data = {
    "inputs": f"data/mujoco-v1/{controller}/train_inputs.npy",
    "targets": f"data/mujoco-v1/{controller}/train_targets.npy",
}
val_data = {
    "inputs": f"data/mujoco-v1/{controller}/val_inputs.npy",
    "targets": f"data/mujoco-v1/{controller}/val_targets.npy",
}

from utils.dataloader import get_dataloader 

train_loader = get_dataloader(
    model  = 'spectral_ssm',
    data = train_data,
    task = 'mujoco-v1',
    controller = controller,
    bsz  = bsz,
    preprocess =  True, #normalize
    shuffle= True,
    sl = sl,
    device = device
)

val_loader = get_dataloader(
    model  = 'spectral_ssm',
    data = val_data,
    task = 'mujoco-v1',
    controller =  controller,
    bsz = bsz,
    preprocess =  True, #normalize
    shuffle= True,
    sl = sl,
    device = device
)


Creating dataloader on cuda for task: mujoco-v1
Using sequence length: 512

Apply Gaussian noise to data?: Disabled

Calculating data statistics...
Normalizing data...
Validating data normalization...

Normalized mean of inputs for coordinates: [0.00018377 0.0004239 ]
Normalized standard deviation of inputs for coordinates: [0.99988493 1.00014359]
Normalized mean of targets for coordinates: [-0.00018377 -0.0004239 ]
Normalized standard deviation of targets for coordinates: [1.00011158 0.99980374]

Normalized mean of inputs for angles: [-0.00020589 -0.00096165 -0.00018324  0.00073418 -0.00010059 -0.00011929
 -0.00011729]
Normalized standard deviation of inputs for angles: [1.00041433 1.00185566 1.00013121 1.00021316 1.00018604 1.00035338
 1.00017155]
Normalized mean of targets for angles: [ 0.00020589  0.00096165  0.00018324 -0.00073418  0.00010059  0.00011929
  0.00011729]
Normalized standard deviation of targets for angles: [0.99954492 0.9977162  0.9997683  0.9997608  0.99939967 0.99

In [193]:
training_stu = True #@param
num_epochs: int = 1 #@param
steps_per_epoch = len(train_loader) #@param
num_steps: int = steps_per_epoch * num_epochs #@param
dilation: int = 1 #@param
warmup_steps: int = num_steps // 8 #@param
eval_period: int = num_steps // 16 #@param

print(f"\nUsing batch size: {bsz}")
print(f"Number of epochs: {num_epochs}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"=> Number of training steps: {num_steps}")


Using batch size: 1
Number of epochs: 1
Steps per epoch: 2811
=> Number of training steps: 2811


In [194]:
from datetime import datetime
from utils.colors import Colors, colored_print
from losses.loss_ant import AntLoss
from losses.loss_cheetah import HalfCheetahLoss
from losses.loss_walker import Walker2DLoss
from losses.loss_cartpole import CartpoleLoss
import os
import numpy as np

checkpoint_dir: str = "checkpoints"
if controller == "Ant-v1":
    loss_fn = AntLoss()
elif controller == "HalfCheetah-v1":
    loss_fn = HalfCheetahLoss()
elif controller == "Walker2D-v1":
    loss_fn = Walker2DLoss()
elif controller == "CartPole-v1":
    loss_fn = CartpoleLoss()
else:
    loss_fn = None

In [195]:
patient_counter = 0
best_val_loss = float("inf")
best_model_step = 0
best_checkpoint = None
patience: int = 10 #number of  non-improving eval periods before early stoppig

train_losses = []
val_losses = []
val_time_steps = []
grad_norms = []

model  = model.to(device)

In [205]:
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
for epoch in range(num_epochs):
    for step, (inputs, targets) in enumerate(train_loader):
        inputs,  targets = inputs.to(device), targets.to(device)
        relative_step = epoch * steps_per_epoch + step
        last_step = relative_step == num_steps - 1
        # print(inputs.shape,  targets.shape)
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss,  metrics = loss_fn(outputs, targets)  # Assuming `loss_function` is defined
        train_losses.append(loss.item())

        # Backward pass and optimization step
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip global norm of gradient at 1.0, per the GPT-3 paper
        grad_norms.append(grad_norm.item())
        optimizer.step()

        # Periodically evaluate the model on validation set
        if relative_step % (eval_period // dilation) == 0 or last_step:
            colored_print(
                f"\nEvaluating the spectral SSM model at step {relative_step}.",
                Colors.OKCYAN,
            )
            model.eval()
            with torch.no_grad():
                val_loss = 0.0
                for val_inputs, val_targets in val_loader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                    val_outputs = model(val_inputs)
                    loss, metrics = loss_fn(val_outputs, val_targets) 
                    val_loss += loss.item()
                val_loss /= len(val_loader)
                val_losses.append(val_loss)
                val_time_steps.append(relative_step)
            model.train()

            colored_print(
                f"\nValidation Loss: {val_loss:.4f}.",
                Colors.OKCYAN,
            )

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_step = relative_step
                patient_counter = 0

                # Save model and optimizer state
                model_checkpoint = f"sssm-{controller}-model_step-{relative_step}-{timestamp}.pt"
               
                model_path = os.path.join(checkpoint_dir, model_checkpoint)
                torch.save(model.state_dict(), model_path)

                extra_info = f"sssm-{controller}-other_step-{relative_step}-{timestamp}.pt"
                best_checkpoint =model_checkpoint, extra_info
                extra_info_path = os.path.join(checkpoint_dir, extra_info)
                torch.save(
                    {
                        "optimizer": optimizer.state_dict(),
                        "configs": model.configs,
                        "step": relative_step,
                        "val_loss": val_loss,
                        "timestamp": timestamp,
                        "rng_state_pytorch": torch.get_rng_state(),
                        "rng_state_numpy": np.random.get_state(),
                        "rng_state_cuda": torch.cuda.get_rng_state_all()
                        if torch.cuda.is_available()
                        else None,
                    },
                    extra_info_path,
                )
                colored_print(
                    f"New best validation loss: {val_loss:.4f}. Model checkpoint saved at {model_path}.",
                    Colors.OKGREEN,
                )
            else:
                patient_counter += 1
                colored_print(
                    f"No improvement in validation loss. Current best: {best_val_loss:.4f}.",
                    Colors.WARNING,
                )

            if patient_counter >= patience:
                colored_print(
                    f"Stopping early due to patience limit of {patience}.",
                    Colors.FAIL,
                )
                break

        # Logging
        if relative_step % 10 == 0:
            colored_print(f"\nStep {relative_step:5d}", Colors.HEADER)
            colored_print(f"Loss: {loss.item():.6f} | Gradient Norm: {grad_norm:.4f}", Colors.OKBLUE)

    if patient_counter >= patience:
        break



Evaluating the spectral SSM model at step 0.

Validation Loss: 0.8502.
No improvement in validation loss. Current best: 0.8501.

Step     0
Loss: 0.765131 | Gradient Norm: 0.1048

Step    10
Loss: 0.740599 | Gradient Norm: 0.1574

Step    20
Loss: 0.887945 | Gradient Norm: 0.3049

Step    30
Loss: 0.934229 | Gradient Norm: 0.2788

Step    40
Loss: 0.754667 | Gradient Norm: 0.1731

Step    50
Loss: 0.808573 | Gradient Norm: 0.1272

Step    60
Loss: 0.998579 | Gradient Norm: 0.3215

Step    70
Loss: 0.949057 | Gradient Norm: 0.1529

Step    80
Loss: 0.791073 | Gradient Norm: 0.2070

Step    90
Loss: 0.742288 | Gradient Norm: 0.1725

Step   100
Loss: 0.791478 | Gradient Norm: 0.1243

Step   110
Loss: 1.115946 | Gradient Norm: 0.2458

Step   120
Loss: 0.971060 | Gradient Norm: 0.1712

Step   130
Loss: 0.737454 | Gradient Norm: 0.1752

Step   140
Loss: 0.897802 | Gradient Norm: 0.1991

Step   150
Loss: 0.708089 | Gradient Norm: 0.2631

Step   160
Loss: 0.816766 | Gradient Norm: 0.1230

Ste

In [ ]:
from  train import save_results
task = 'mujoco-v1'

if best_checkpoint:
    model_checkpoint, extra_info = best_checkpoint
    best_model_path = os.path.join(checkpoint_dir, model_checkpoint)
    best_model_extra_info_path = os.path.join(checkpoint_dir, extra_info)

    # Load the best checkpoint
    state_dict = torch.load(best_model_path, map_location="cpu")
    model.load_state_dict(state_dict)

    # Load optimizer and other data
    other_data = torch.load(best_model_extra_info_path, map_location="cpu")
    optimizer.load_state_dict(other_data["optimizer"])

    # Display best model information
    print("\nBest model information for the spectral SSM model:")
    print(f"    Best model at step {best_model_step}")
    print(f"    Best validation loss: {best_val_loss:.4f}")
    print(f"    Best model checkpoint saved at: {best_model_path}")
    print(f"    Additional data saved at: {best_model_extra_info_path}")

    # Save training details to a file
    training_details_path = f"training_details_sssm_{timestamp}.txt"
    with open(training_details_path, "w") as f:
        f.write(f"Training completed for the spectral SSM on {task} at: {datetime.now()}\n")
        f.write(f"Best model step: {best_model_step}\n")
        f.write(f"Best validation loss: {best_val_loss:.4f}\n")
        f.write(f"Best model checkpoint saved at: {best_model_path}\n")
        f.write(f"Additional data saved at: {best_model_extra_info_path}\n")

    print(f"Training details saved in {training_details_path}. Congratulations!")
else:
    print("\nNo best checkpoint found. The model did not improve during training.")

# Save final results

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)
if not os.path.exists("results/"):
    os.makedirs("results/")
if not os.path.exists("landscapes"):
    os.makedirs("landscapes", exist_ok=True)

save_results(task, controller, train_losses, "train_losses", timestamp)
save_results(task, controller,val_losses, "val_losses", timestamp)
save_results(task, controller, val_time_steps,"val_time_steps", timestamp)
save_results(task, controller, grad_norms, "grad_norms", timestamp)

print("Training run completed. Results have been saved.")

# from utils.loss_landscape import LossLandscape (takes 20 minutes to run)
# #Don't exactly know how to visualize thi
# loss_landscape = LossLandscape(model, device, optimizer, lr)
# x_range = (-1, 1, 10)  # Adjust as needed
# y_range = (-1, 1, 10)
# loss_landscape.generate(
#     train_loader,
#     f"landscapes/loss_landscape-{timestamp}",
#     x_range=x_range,
#     y_range=y_range,
#     plot_loss_landscape=True,
#     plot_hessian=True,
# )


C:\Users\devan\AppData\Local\Temp\ipykernel_42600\1417480683.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(best_model_path, map_location="cpu"


Best model information for the spectral SSM model:
    Best model at step 2275
    Best validation loss: 0.8492
    Best model checkpoint saved at: checkpoints\sssm-Walker2D-v1-model_step-2275-2024-11-18-16-34-38.pt
    Additional data saved at: checkpoints\sssm-Walker2D-v1-other_step-2275-2024-11-18-16-34-38.pt
Training details saved in training_details_sssm_2024-11-18-16-34-38.txt. Congratulations!
Data saved to results\mujoco-v1\sssm\sssm-Walker2D-v1-train_losses-2024-11-18-16-34-38.txt
Data saved to results\mujoco-v1\sssm\sssm-Walker2D-v1-val_losses-2024-11-18-16-34-38.txt
Data saved to results\mujoco-v1\sssm\sssm-Walker2D-v1-val_time_steps-2024-11-18-16-34-38.txt
Data saved to results\mujoco-v1\sssm\sssm-Walker2D-v1-grad_norms-2024-11-18-16-34-38.txt
Training run completed. Results have been saved.


c:\Users\devan\OneDrive\Documents\mujoco\sssm_basic\utils\loss_landscape.py:351: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_data = torch.load(landscape_data)
c:\Use

VTS file with grid saved to landscapes/loss_landscape-2024-11-18-16-34-38.vts
Loss landscape visualization saved to landscapes/loss_landscape-2024-11-18-16-34-38.png


c:\Users\devan\OneDrive\Documents\mujoco\sssm_basic\utils\loss_landscape.py:635: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_data = torch.load(landscape_data)


Hessian eigenvalue ratio heatmap saved to landscapes/loss_landscape-2024-11-18-16-34-38_hessian_heatmap.png
Percentage of convex regions: 100.00%
Percentage of near-convex regions (ratio >= -0.01): 100.00%


In [207]:
model

SSSM(
  (loss_fn): MSELoss()
  (emb): Linear(in_features=24, out_features=8, bias=False)
  (stu): ModuleDict(
    (dropout): Dropout(p=0.0, inplace=False)
    (hidden): ModuleList(
      (0-1): 2 x Block(
        (rn_1): RMSNorm((8,), eps=None, elementwise_affine=True)
        (stu): STU(
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (rn_2): RMSNorm((8,), eps=None, elementwise_affine=True)
        (mlp): MLP(
          (swiglu): SwiGLU(
            (w): Linear(in_features=8, out_features=32, bias=False)
            (v): Linear(in_features=8, out_features=32, bias=False)
            (w2): Linear(in_features=32, out_features=8, bias=False)
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
  )
  (task_head): Linear(in_features=8, out_features=18, bias=False)
)